### dev_notebook_

This is your development environment...you may start:
- Exploring your data.
- Testing the ad-hoc methods.
- Designing your pipeline.

In [1]:
import pandas as pd
import requests as re

In [16]:
#LEEMOS NUESTRO FICHERO
bicimad_df = pd.read_csv('../data/bicimad_stations.csv', sep = '\t')

In [3]:
bicimad_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            264 non-null    int64 
 1   id                    264 non-null    int64 
 2   name                  264 non-null    object
 3   light                 264 non-null    int64 
 4   number                264 non-null    object
 5   address               264 non-null    object
 6   activate              264 non-null    int64 
 7   no_available          264 non-null    int64 
 8   total_bases           264 non-null    int64 
 9   dock_bikes            264 non-null    int64 
 10  free_bases            264 non-null    int64 
 11  reservations_count    264 non-null    int64 
 12  geometry.type         264 non-null    object
 13  geometry.coordinates  264 non-null    object
dtypes: int64(9), object(5)
memory usage: 29.0+ KB


In [4]:
bicimad_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            264 non-null    int64 
 1   id                    264 non-null    int64 
 2   name                  264 non-null    object
 3   light                 264 non-null    int64 
 4   number                264 non-null    object
 5   address               264 non-null    object
 6   activate              264 non-null    int64 
 7   no_available          264 non-null    int64 
 8   total_bases           264 non-null    int64 
 9   dock_bikes            264 non-null    int64 
 10  free_bases            264 non-null    int64 
 11  reservations_count    264 non-null    int64 
 12  geometry.type         264 non-null    object
 13  geometry.coordinates  264 non-null    object
dtypes: int64(9), object(5)
memory usage: 29.0+ KB


In [5]:
bicimad_df.head(1)

,Unnamed: 0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"


In [6]:
'''
geometry = bicimad_df['geometry.coordinates'][0]
geometry = geometry.strip('[]')
coordenadas = geometry.split(', ')
print(coordenadas)
longitud = float(coordenadas[0])
latitud = float(coordenadas[1])
print(longitud)
print(latitud)
'''

"\ngeometry = bicimad_df['geometry.coordinates'][0]\ngeometry = geometry.strip('[]')\ncoordenadas = geometry.split(', ')\nprint(coordenadas)\nlongitud = float(coordenadas[0])\nlatitud = float(coordenadas[1])\nprint(longitud)\nprint(latitud)\n"

In [7]:
bicimad_df['longitude'] = bicimad_df.apply(lambda row: row['geometry.coordinates'].strip('[]').split(', ')[0], axis=1)

In [8]:
bicimad_df['latitude'] = bicimad_df.apply(lambda row: row['geometry.coordinates'].strip('[]').split(', ')[1], axis=1)

In [9]:
bicimad_df.head(2)

,Unnamed: 0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,longitude,latitude
0,0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",-3.7018341,40.4172137
1,1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]",-3.701602938060457,40.41731271011562


In [10]:
bicipark_df = pd.read_csv('../data/bicipark_stations.csv', sep=';')

In [ ]:
bicipark_df['longitude'] = bicipark_df.apply(lambda row: row['geometry.coordinates'].strip('[]').split(', ')[0], axis=1)
bicipark_df['latitude'] = bicipark_df.apply(lambda row: row['geometry.coordinates'].strip('[]').split(', ')[1], axis=1)

In [13]:
#bicipark_df[['longitude', 'latitude']] = bicipark_df.apply(lambda row: pd.Series([row['geometry.coordinates'].strip('[]').split(', ')[0], row['geometry.coordinates'].strip('[]').split(', ')[1]]), axis=1)

In [12]:
bicipark_df.head()

,Unnamed: 0,stationId,stationName,address,city,zip_code,total_places,free_places,enabled,reserved_places,geometry.type,geometry.coordinates,longitude,latitude
0,0,74,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,28050,10,9,1,0,Point,"[-3.6630679, 40.4846838]",-3.6630679,40.4846838
1,1,22,Bicipark Orense,"Avenida General Perón, 27",Madrid,28020,15,9,1,1,Point,"[-3.693897, 40.452973]",-3.693897,40.452973
2,2,49,Bicipark Montalban,"Calle de Montalbán, 5",Madrid,28014,16,15,1,0,Point,"[-3.690786, 40.418146]",-3.690786,40.418146
3,3,12,Bicipark Almagro,"Calle Almagro, 11",Madrid,28010,16,12,1,0,Point,"[-3.692879, 40.430361]",-3.692879,40.430361
4,4,9,Bicipark Recoletos,"Paseo de Recoletos, 2",Madrid,28001,10,3,0,0,Point,"[-3.692245, 40.420354]",-3.692245,40.420354


In [14]:
#CONECTAMOS CON LA WEB PARA OBTENER JSON
base_url = "https://datos.madrid.es/egob"
body = "/catalogo/209434-0-templos-otros.json"
response = re.get(base_url + body)
print(response)

<Response [200]>


In [15]:
#ANALIZAMOS JSON
content = response.content
json_data = response.json()
print(json_data)

{'@context': {'c': 'http://www.w3.org/2002/12/cal#', 'dcterms': 'http://purl.org/dc/terms/', 'geo': 'http://www.w3.org/2003/01/geo/wgs84_pos#', 'loc': 'http://purl.org/ctic/infraestructuras/localizacion#', 'org': 'http://purl.org/ctic/infraestructuras/organizacion#', 'vcard': 'http://www.w3.org/2006/vcard/ns#', 'schema': 'https://schema.org/', 'title': 'vcard:fn', 'id': 'dcterms:identifier', 'relation': 'dcterms:relation', 'references': 'dcterms:references', 'address': 'vcard:adr', 'area': 'loc:barrio', 'district': 'loc:distrito', 'locality': 'vcard:locality', 'postal-code': 'vcard:postal-code', 'street-address': 'vcard:street-address', 'location': 'vcard:geo', 'latitude': 'geo:lat', 'longitude': 'geo:long', 'organization': 'vcard:org', 'organization-desc': 'dcterms:description', 'accesibility': 'org:accesibilidad', 'services': 'org:servicios', 'schedule': 'org:horario', 'organization-name': 'vcard:organization-name', 'description': 'c:summary', 'link': 'c:url', 'uid': 'c:uid', 'dtstar

In [17]:
json_data.keys()

dict_keys(['@context', '@graph'])

In [21]:
df1 = pd.DataFrame(json_data['@graph'])

In [26]:
df1.location[0]
df1.head()

,@id,@type,id,title,relation,address,location,organization
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11772877,Asamblea Cristiana Gilgal,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.38724931072868, 'longitude': -...",{'organization-desc': 'Templo evangélico. Bus...
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11025958,Asambleas de Evangelización Mundial para Crist...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.372800292475574, 'longitude': ...",{'organization-desc': 'Templo evangélico Metr...
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11026848,Asociación Evangélica Internacional,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.457895405861926, 'longitude': ...",{'organization-desc': 'Templo evangélico. Met...
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,3387,Catedral Anglicana de Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.42635538209311, 'longitude': -...",{'organization-desc': 'Iglesia Española Reform...
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,3388,Catedral Ortodoxa Griega. Santa Iglesia Catedr...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.45755529500435, 'longitude': -...",{'organization-desc': ' Metro: Colombia (líne...


In [ ]:
df1[['longitude', 'latitude']] = df1.apply(lambda row: pd.Series([row['geometry.coordinates'].strip('[]').split(', ')[0], row['geometry.coordinates'].strip('[]').split(', ')[1]]), axis=1)

In [25]:
latitude = df1['latitude']
longitude = df1['longitude']

KeyError: 'latitude'